In [104]:
# Installations of libraries
!pip install PyPDF2
!pip install nltk
!pip install pdfminer
!pip install tensorflow

In [105]:
# nltk download
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [106]:
# Imports
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

import io
import nltk

In [107]:
# Initiate pdf file
pdf_file = "../res/who_int_0.pdf"
# Initiate dump file
temp_text = "../dump/temp.txt"

In [108]:
# https://stackoverflow.com/questions/64467930/python-pypdf2-extracttext-not-working
inFile = open(pdf_file, 'rb')
resMgr = PDFResourceManager()
retData = io.StringIO()
TxtConverter = TextConverter(resMgr, retData,laparams = LAParams())
interpreter = PDFPageInterpreter(resMgr, TxtConverter)

# Process each pages
for page in PDFPage.get_pages(inFile): 
    interpreter.process_page(page)

# Write to text temporary file
txt = retData.getvalue()
with open(temp_text, 'w') as f: 
     f.write(txt)

In [175]:
raw = open(outTXTFile).read()

In [176]:
# Length and Type of raw text file
print(len(raw), type(raw))

7876 <class 'str'>


In [177]:
# Initialize Tokenizer
tokens = nltk.word_tokenize(raw)

In [178]:
# Length of tokenized text file
print(len(tokens), type(tokens))

1353 <class 'list'>


In [179]:
# Sample print of list
print(tokens[0:69])

['3/15/22', ',', '10:32', 'AM', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', 'Western', 'Pacific', 'Philippines', '©', 'Opinion', ':', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', '11', 'March', '2022', 'Statement', '|', 'By', 'Dr', 'Rajendra', 'Yadav', 'and', 'Dr', 'Sangjun', 'Moon', ',', 'WHO', 'Philippines', 'Manila', ',', 'Philippines', '(', '11', 'March', '2022', ')', '–', 'When', 'will', 'this', 'end', '?', 'This', 'question', 'has', 'been', 'on', 'everyone', '’', 's', 'mind', 'for', 'many', 'months', '.', 'We', 'are', 'all', 'tired']


In [235]:
it = iter(tokens)
size = [i for i in range(len(tokens))]
token_dict = dict(zip((size), it))

In [236]:
token_dict

{0: '3/15/22',
 1: ',',
 2: '10:32',
 3: 'AM',
 4: 'Is',
 5: 'the',
 6: 'pandemic',
 7: 'ending',
 8: 'soon',
 9: '?',
 10: 'Western',
 11: 'Pacific',
 12: 'Philippines',
 13: '©',
 14: 'Opinion',
 15: ':',
 16: 'Is',
 17: 'the',
 18: 'pandemic',
 19: 'ending',
 20: 'soon',
 21: '?',
 22: '11',
 23: 'March',
 24: '2022',
 25: 'Statement',
 26: '|',
 27: 'By',
 28: 'Dr',
 29: 'Rajendra',
 30: 'Yadav',
 31: 'and',
 32: 'Dr',
 33: 'Sangjun',
 34: 'Moon',
 35: ',',
 36: 'WHO',
 37: 'Philippines',
 38: 'Manila',
 39: ',',
 40: 'Philippines',
 41: '(',
 42: '11',
 43: 'March',
 44: '2022',
 45: ')',
 46: '–',
 47: 'When',
 48: 'will',
 49: 'this',
 50: 'end',
 51: '?',
 52: 'This',
 53: 'question',
 54: 'has',
 55: 'been',
 56: 'on',
 57: 'everyone',
 58: '’',
 59: 's',
 60: 'mind',
 61: 'for',
 62: 'many',
 63: 'months',
 64: '.',
 65: 'We',
 66: 'are',
 67: 'all',
 68: 'tired',
 69: 'of',
 70: 'the',
 71: 'pandemic',
 72: '.',
 73: 'So',
 74: 'naturally',
 75: ',',
 76: 'we',
 77: 'all',
 

In [261]:
df = pd.DataFrame(list(zip(size, tokens)), columns=["Token_ID", "Token"])
print(df.to_string(index=False))

 Token_ID                                                                                Token
        0                                                                              3/15/22
        1                                                                                    ,
        2                                                                                10:32
        3                                                                                   AM
        4                                                                                   Is
        5                                                                                  the
        6                                                                             pandemic
        7                                                                               ending
        8                                                                                 soon
        9                                         

In [262]:
df.to_csv ('../data/data.csv', index = False, header=True)